In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Dense, GlobalAvgPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet152V2, InceptionV3
from google.cloud import storage
from google.colab import drive
import wandb
from wandb.integration.keras import WandbCallback


# Model Testing

## First step

Download the tensorized tfrecord from GCP

In [ ]:
def download_tesorized_data_from_bucket(secrets_path, bucket_name, local_dir):
    client = storage.Client.from_service_account_json(secrets_path)
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    os.makedirs(local_dir, exist_ok=True)

    for blob in blobs:
        local_path = os.path.join(local_dir, blob.name)
        local_dir_path = os.path.dirname(local_path)
        os.makedirs(local_dir_path, exist_ok=True)

        blob.download_to_filename(local_path)
        print(f"Downloaded {blob.name} to {local_path}")

Set up secret variables and GCP credentials

In [ ]:
SECRETS_PATH = '/content/data-service-account-model.json'
BUCKET_NAME = 'tensor-bucket-20k'
LOCAL_PATH = '/content/tensor'
download_tesorized_data_from_bucket(SECRETS_PATH, BUCKET_NAME, LOCAL_PATH)

Downloaded data.tfrecord to /content/tensor/data.tfrecord


In [ ]:
LOCAL_TFRECORD_PATH = '/content/tensor/data.tfrecord'
raw_dataset = tf.data.TFRecordDataset(LOCAL_TFRECORD_PATH)

## Second Step

Parse the downloaded tfrecord and prepare the data for later model testing

In [ ]:
feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}

In [ ]:
def parse_tfrecord(LOCAL_TFRECORD_PATH):
    # Parse the tensorized data using the feature description
    parsed_example = tf.io.parse_single_example(LOCAL_TFRECORD_PATH, feature_description)

    # Decode the raw bytes to get the image
    image = tf.io.decode_raw(parsed_example['image_raw'], tf.uint8)
    image = tf.reshape(image, [IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]

    # Get the label
    label = parsed_example['label']
    return image, label

In [ ]:
LOCAL_TFRECORD_PATH = '/content/tensor/data.tfrecord'
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CHANNELS = 3

In [ ]:
parsed_dataset = raw_dataset.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

Determine the number of classes (different car model, year, and make) in the data

In [ ]:
# Determine the number of classes
labels = []
for _, label in parsed_dataset:
    labels.append(label.numpy())
unique_labels = set(labels)
NUM_CLASSES = len(unique_labels)
print(f"Number of classes: {NUM_CLASSES}")

# One-hot encode labels if using categorical crossentropy
def one_hot_encode(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

parsed_dataset = parsed_dataset.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)

Number of classes: 196


Perform random train-validation split on the data

In [ ]:
# Shuffle and split the dataset into training and validation sets
dataset = parsed_dataset.shuffle(buffer_size=10000)
dataset_size = len(labels)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Batch and prefetch the datasets
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
type(train_dataset)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

## Third Step

Finetune the model with ResNet152V2 weights with 3 layers in the fully-connected layers and Leaky ReLu as the activation function

In [ ]:
# Model Name
name1 = "CarNetV1"

# Pretrained Model
base_model = ResNet152V2(include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS), weights='imagenet')
base_model.trainable = False # Freeze the Weights

# Model
CarNetV1 = Sequential([
    base_model,
    GlobalAvgPool2D(),
    Dense(224, activation='leaky_relu'),
    Dense(NUM_CLASSES, activation='softmax')
], name=name1)

CarNetV1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [ ]:
# Train
EPOCHS = 100
start_time = time.time()
history = CarNetV1.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)
execution_time = (time.time() - start_time) / 60.0
print(f"Training completed in {execution_time:.2f} minutes")

Epoch 1/100
    500/Unknown 111s 167ms/step - accuracy: 0.0932 - loss: 4.4468

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 2.48677, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 146s 236ms/step - accuracy: 0.0933 - loss: 4.4455 - val_accuracy: 0.3882 - val_loss: 2.4868
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.4041 - loss: 2.3882
Epoch 2: val_loss improved from 2.48677 to 1.69286, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 116s 228ms/step - accuracy: 0.4042 - loss: 2.3880 - val_accuracy: 0.5742 - val_loss: 1.6929
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.5673 - loss: 1.6414
Epoch 3: val_loss improved from 1.69286 to 1.22908, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 117s 231ms/step - accuracy: 0.5673 - loss: 1.6414 - val_accuracy: 0.6858 - val_loss: 1.2291
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.6940 - loss: 1.1532
Epoch 4: val_loss improved from 1.22908 to 0.88513, saving model to best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 118s

In [ ]:
# Evaluate CarNetV1
val_loss, val_accuracy = CarNetV1.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 187ms/step - accuracy: 0.9763 - loss: 0.0586
Validation Loss: 0.05348135158419609
Validation Accuracy: 0.9807500243186951


Finetune the model with ResNet152V2 weights with 4 layers in the fully-connected layers and ReLu as the activation function

In [ ]:
# Model Name
name2 = "CarNetV2"

# Pretrained Model
base_model = ResNet152V2(include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS), weights='imagenet')
base_model.trainable = False # Freeze the Weights

# Model
CarNetV2 = Sequential([
    base_model,
    GlobalAvgPool2D(),
    Dense(448, activation='relu'),
    Dense(224, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
], name=name2)

CarNetV2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model2.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

In [ ]:
# Train
EPOCHS = 100
start_time = time.time()
history = CarNetV2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)
execution_time = (time.time() - start_time) / 60.0
print(f"Training completed in {execution_time:.2f} minutes")

Epoch 1/100
    500/Unknown 117s 188ms/step - accuracy: 0.0596 - loss: 4.6207

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 3.05169, saving model to best_model2.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 154s 262ms/step - accuracy: 0.0597 - loss: 4.6196 - val_accuracy: 0.2362 - val_loss: 3.0517
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.2605 - loss: 2.9535
Epoch 2: val_loss improved from 3.05169 to 2.32645, saving model to best_model2.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 125s 246ms/step - accuracy: 0.2605 - loss: 2.9532 - val_accuracy: 0.3938 - val_loss: 2.3265
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.4019 - loss: 2.2529
Epoch 3: val_loss improved from 2.32645 to 1.77350, saving model to best_model2.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 244ms/step - accuracy: 0.4019 - loss: 2.2528 - val_accuracy: 0.5228 - val_loss: 1.7735
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.5222 - loss: 1.7553
Epoch 4: val_loss improved from 1.77350 to 1.40754, saving model to best_model2.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Evaluate CarNetV2
val_loss, val_accuracy = CarNetV2.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 206ms/step - accuracy: 0.9672 - loss: 0.1022


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Validation Loss: 0.11648868769407272
Validation Accuracy: 0.9647499918937683


Finetune the model with InceptionV3 weights with 3 layers in the fully-connected layers and Leaky ReLu as the activation function

In [ ]:
# Model Name
name3 = "CarNetV3"

# Pretrained Model
base_model = InceptionV3(include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS), weights='imagenet')
base_model.trainable = False # Freeze the Weights

# Model
CarNetV3 = Sequential([
    base_model,
    GlobalAvgPool2D(),
    Dense(224, activation='leaky_relu'),
    Dense(NUM_CLASSES, activation='softmax')
], name=name3)

CarNetV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model3.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Train
EPOCHS = 100
start_time = time.time()
history = CarNetV3.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)
execution_time = (time.time() - start_time) / 60.0
print(f"Training completed in {execution_time:.2f} minutes")

Epoch 1/100
    500/Unknown 49s 65ms/step - accuracy: 0.0596 - loss: 4.6848

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 3.36879, saving model to best_model3.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 66s 99ms/step - accuracy: 0.0597 - loss: 4.6838 - val_accuracy: 0.2020 - val_loss: 3.3688
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2283 - loss: 3.1954
Epoch 2: val_loss improved from 3.36879 to 2.64928, saving model to best_model3.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 90ms/step - accuracy: 0.2283 - loss: 3.1953 - val_accuracy: 0.3392 - val_loss: 2.6493
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3486 - loss: 2.5855
Epoch 3: val_loss improved from 2.64928 to 2.15760, saving model to best_model3.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 46s 88ms/step - accuracy: 0.3486 - loss: 2.5855 - val_accuracy: 0.4442 - val_loss: 2.1576
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.4480 - loss: 2.1485
Epoch 4: val_loss improved from 2.15760 to 1.83141, saving model to best_model3.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 90ms/

In [ ]:
# Evaluate CarNetV3
val_loss, val_accuracy = CarNetV3.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 64ms/step - accuracy: 0.9607 - loss: 0.1217


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Validation Loss: 0.1162281259894371
Validation Accuracy: 0.9627500176429749


We decided to move on with CarNetV3 which gives a high validation accuracy and shortest training time.